In [1]:
#%load_ext tensorboard
import numpy as np
#!pip install torch torchvision
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as T
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset
from torch.utils.data.distributed import DistributedSampler
from torch.optim import SGD
from torch.optim.lr_scheduler import StepLR

import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint

import pandas as pd

from PIL import Image, ImageFile

import matplotlib.pyplot as plt

import torchvision.models as models

from tqdm import tqdm

from sklearn.model_selection import train_test_split

import os
import getpass
import sys

/home/users/lameski/.local/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/users/lameski/.local/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/users/lameski/.local/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/users/lameski/.local/lib/python3.6/site-pac

In [2]:
# X = np.asarray([[0,1],[1,1],[4,0],[5,0],[6,0],[7,0],[8,1],[9,0],[10,0]])
# X_ = X.T
# X_train, y_train = train_test_split(X, test_size=0.33,stratify = X_[1])
# print(X_train)
# print(y_train)

In [3]:
torch.cuda.get_device_name(0)

'GeForce GTX TITAN X'

In [4]:
username = getpass.getuser()

train_path = f'/home/users/{username}/kaggle-data/jpeg/train'
train_meta = f'/home/users/{username}/kaggle-data/train.csv'

test_path = f'/home/users/{username}/kaggle-data/jpeg/test'
test_meta = f'/home/users/{username}/kaggle-data/test.csv'



classes = 2

In [5]:
class KaggleDataset(torch.utils.data.Dataset):
    
    def __init__(self, img_path, meta_path, test_path, test_meta_path,tr = True, test= False):
        super(KaggleDataset, self).__init__()
        self.img_path = img_path
        self.meta_path = meta_path
        self.meta = pd.read_csv(self.meta_path)
        self.tr = tr
        self.test = test
        self.train, self.val = train_test_split(self.meta, test_size = 0.2, stratify = self.meta.target)
        if test:
            self.test_path = test_path
            self.test_meta_path = test_meta_path
            self.test_meta = pd.read_csv(self.test_meta_path)
            
    def __len__(self):
        if self.tr and not self.test:
            return len(self.train)
        elif not self.tr and not self.test:
            return len(self.val)
        else: 
            return len(self.test_meta)
    
    def __getitem__(self, idx):
        
        if self.tr and not self.test:
            label = self.train.iloc()[idx]

            target = label.target

            path = os.path.join(self.img_path, label.image_name + ".jpg")
            input_image = Image.open(path)
            preprocess = T.Compose([
                T.Resize((224,224)),
                #T.CenterCrop(224),
                T.ToTensor(),
                T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            ])
            input_tensor = preprocess(input_image)
            #input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model
            if input_tensor.shape[0]==1:
                input_tensor = input_tensor.repeat(3,1,1)
            return input_tensor, target
        elif not self.tr and not self.test:
            label = self.val.iloc()[idx]

            target = label.target

            path = os.path.join(self.img_path, label.image_name + ".jpg")
            input_image = Image.open(path)
            preprocess = T.Compose([
                T.Resize((224,224)),
                #T.CenterCrop(224),
                T.ToTensor(),
                T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            ])
            input_tensor = preprocess(input_image)
            #input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model
            if input_tensor.shape[0]==1:
                input_tensor = input_tensor.repeat(3,1,1)
            return input_tensor, target
        
        else:
            label = self.test_meta.iloc()[idx]

#             target = self.label.index(meta.target)

            path = os.path.join(self.test_path, label.image_name + ".jpg")
            input_image = Image.open(path)
            preprocess = T.Compose([
                T.Resize((224,224)),
                #T.CenterCrop(224),
                T.ToTensor(),
                T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            ])
            input_tensor = preprocess(input_image)
            #input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model
            if input_tensor.shape[0]==1:
                input_tensor = input_tensor.repeat(3,1,1)
            
            return input_tensor, label.image_name#self.test_meta['image_name'].tolist()

In [6]:
# self.train, self.val = train_test_split(meta, test_size = 0.2, stratify = meta.target)
def get_dataset(img_path, meta_path,test_path, test_meta_path, train, test):
    dataset = KaggleDataset(img_path, meta_path,test_path, test_meta_path, train, test)
    return dataset

def get_loader(dataset, shuffle, size=128):
    loader = DataLoader(dataset, batch_size=size,
                        shuffle=shuffle, num_workers=8)
    return loader

def build_dataloader(img_path, meta_path,test_path, test_meta_path,):
 
    train_dataset = get_dataset(img_path, meta_path,test_path, test_meta_path,
                                True, False)
    val_dataset = get_dataset(img_path, meta_path,test_path, test_meta_path,
                              False, False)
    test_dataset = get_dataset(img_path, meta_path,test_path, test_meta_path,
                               True, True)
    
    train_loader = get_loader(train_dataset, True)
    val_loader = get_loader(val_dataset, False)
    test_loader = get_loader(test_dataset, False)
    
    
    return train_loader, val_loader, test_loader


In [7]:
# resnet.fc
# Linear(in_features=2048, out_features=1000, bias=True)

In [8]:
class Resnet(pl.LightningModule):
    

    def __init__(self, freeze=True, lr=0.01, names=None):
        super(Resnet, self).__init__()
        self.resnet = models.resnext50_32x4d(pretrained=True)
        
        if freeze is True:
            # freeze feature layers
            for param in self.resnet.parameters():
                param.requires_grad = False
        
        # adjust classes
        self.resnet.fc = nn.Linear(2048, classes)
        
        self.loaders = None
        self.lr = lr
        self.names = names
            
    def forward(self, inp):
        return self.resnet(inp)
    
    def training_step(self, batch, batch_nb):
        x, y = batch
        loss = F.cross_entropy(self(x), y)
        tensorboard_logs = {'train_loss' : loss}
        return {'loss': loss, 'log': tensorboard_logs}
    
    def configure_optimizers(self):
        optimizer = SGD(self.parameters(),
                       lr=self.lr,
                       momentum=0.9,
                       weight_decay=0.0005)
        scheduler = StepLR(optimizer, 1)
        return [optimizer], [scheduler]
    
    def validation_step(self, batch, batch_nb):
        x, y = batch
        loss = F.cross_entropy(self(x), y)
        return {'val_loss': loss}
    
    def validation_epoch_end(self, outputs):
        avg_loss = torch.stack([x['val_loss'] for x in outputs]).mean()
        tensorboard_logs = {'val_loss': avg_loss}
        return {'val_loss': avg_loss, 'log': tensorboard_logs}
    
#     def test_step(self, batch, batch_nb):
#         x, y = batch
#         y_hat = self(x).argmax(dim=1)
#         acc = (y == y_hat).float().mean()
#         c_accs = []
#         for i in range(classes):
#             class_elem = y == i
#             c_acc = (y_hat[class_elem] == i).sum().float() / (class_elem.sum() + 1e-9)
#             c_accs.append(c_acc)
#         return {'accuracy' : acc, 'class_accuracies' : c_accs}
#         return y_hat

#     def test_epoch_end(self, outputs):
#         avg_acc = torch.stack([x['accuracy'] for x in outputs]).mean().item()
#         c_accs = [x['class_accuracies'] for x in outputs]
#         avg_c_acc = [torch.stack([l[i] for l in c_accs]).mean().item() for i in range(classes)]
#         if self.names is not None:
#             names = pd.read_pickle(self.names)
#             names = names[0].to_list()
#             avg_c_acc = list(zip(names, avg_c_acc))
#         return {'accuracy' : avg_acc, 'class_accuracies': avg_c_acc}
    
    def build_loaders(self):
        self.loaders = \
            build_dataloader(train_path, train_meta, test_path, test_meta)
    
    def train_dataloader(self):
        if self.loaders is None:
            self.build_loaders()
        return self.loaders[0]
    
    def val_dataloader(self):
        if self.loaders is None:
            self.build_loaders()
        return self.loaders[1]
    
    def test_data(self):
        if self.loaders is None:
            self.build_loaders()
        return self.loaders[2]
    

In [9]:
def run_training(version='v1'):
    checkpoint_callback = ModelCheckpoint(
        filepath='models/resnext50_32x4d.ckpt',
        verbose=True,
    )

    trainer = pl.Trainer( 
        checkpoint_callback=checkpoint_callback,
        gpus=[0],
        distributed_backend='dp', 
        max_nb_epochs=3,
        val_check_interval=0.25,
    )
    
    if version == 'v1':
        model = Resnet()
    else:
        model = Resnet(freeze=False, lr=0.001)
        model.load_state_dict(torch.load('checkpoints/resnext50_32x4dv1.pt'))

    trainer.fit(model)

    torch.save(model.state_dict(), f'checkpoints/resnext50_32x4d{version}.pt')

In [10]:
# # if version == 'v1':
# #         model = Resnet()
# #     else:
# # checkpoint_callback = ModelCheckpoint(
# #         filepath='models/_ckpt_epoch_2.ckpt',
# #         verbose=True,
# #     )
# #model = Resnet()#.load_from_checkpoint(checkpoint_path="models/_ckpt_epoch_2.ckpt")
# # trainer = pl.Trainer( 
# #     checkpoint_callback=checkpoint_callback,
# #     gpus=[0],
# #     distributed_backend='dp', 
# #     max_nb_epochs=1,
# #     val_check_interval=0.25,
# # )
# trainer = pl.Trainer(resume_from_checkpoint='models/_ckpt_epoch_2.ckpt',
#                         gpus=[0],
#                         distributed_backend='dp', 
#                         max_nb_epochs=1,
#                         val_check_interval=0.25)
# #model = Resnet(freeze=False, lr=0.001)
# #model.load_state_dict(torch.load('models/resnext50_32x4dv1.pt'))
# model = Resnet()
# trainer.fit(model)

# torch.save(model.state_dict(), 'models/resnext50_32x4dv1.ckpt')

File "isic_notebook.py", line 309, in run_training
    torch.save(model.state_dict(), f'checkpoints/resnext50_32x4d{version}.pt')
  File "/home/users/lameski/.local/lib/python3.6/site-packages/torch/serialization.py", line 369, in save
    with _open_file_like(f, 'wb') as opened_file:
  File "/home/users/lameski/.local/lib/python3.6/site-packages/torch/serialization.py", line 234, in _open_file_like
    return _open_file(name_or_buffer, mode)
  File "/home/users/lameski/.local/lib/python3.6/site-packages/torch/serialization.py", line 215, in __init__
    super(_open_file, self).__init__(open(name, mode))
FileNotFoundError: [Errno 2] No such file or directory: 'checkpoints/resnext50_32x4dv1.pt'

In [11]:
def evaluate():
    trainer = pl.Trainer(
        gpus=[0],
        distributed_backend='dp')
    model = Resnet()
    model.load_state_dict(torch.load('checkpoints/resnext50_32x4dv1.pt'))
    inp, names = model.test_data()
    model.eval()
    output = model(inp).argmax(dim=1)
    
    with open('output.txt', 'w') as file:
        file.write(str(output))
        
    frame = zip(names, output)
    with open('test_output.txt', 'w') as file:
        file.write(str(frame))

In [20]:
def predict():
    device = torch.device('cuda:0')
    cpu = torch.device('cpu')
    model = Resnet('/models/resnext50_32x4d.pt')
    model.to(device)
    _,_,loader = build_dataloader(train_path, train_meta, test_path, test_meta)
    #loader = model.test_data()
    model.eval()
    preds = []
    
    with torch.no_grad():
        for img, name in tqdm(loader):
            img = img.to(device)
            pred = model(img).argmax(dim=1)
            preds.append([name,pred.to(cpu)])

    #data = [preds[i] for i in range(len(preds))]
    df = pd.DataFrame(preds, columns=['image_name', 'target'])
    df.to_csv('submission.csv') 

In [21]:
#run_training()
predict()

100%|██████████| 86/86 [11:14<00:00,  7.84s/it] 


In [70]:
import re
df = pd.read_csv("submission.csv")
# num1 = df.target.sum()
# overall = len(df.target)
# print(num1, overall)
names = []
targets = []
# for n in df.image_name:
#     #for nn in n:
#     names.append(n)
# for t in df.target:
#     targets.append(t)
#print(re.split(', |,\n        ',df.target[0][8:-2]))#.split(", "))
for i in range(len(df)):
    temp = df.image_name[i][2:-2].split("', '")
    for n in temp:
        names.append(n)
    temp = re.split(', |,\n        ',df.target[i][8:-2])
    for t in temp:
        targets.append(t)

targets = [0 if i=='1' else 1 for i in targets]
#print(names, )
output = pd.DataFrame(list(zip(names, targets)), columns=['image_name', 'target'])
output.to_csv("submission_output.csv")